In [2]:
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage,AverageDollarVolume,Returns
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import Q1500US
from quantopian.pipeline.classifiers.morningstar import Sector
import alphalens as al
import numpy as np

MORNINGSTAR_SECTOR_CODES = {
     -1: 'Misc',
    101: 'Basic Materials',
    102: 'Consumer Cyclical',
    103: 'Financial Services',
    104: 'Real Estate',
    205: 'Consumer Defensive',
    206: 'Healthcare',
    207: 'Utilities',
    308: 'Communication Services',
    309: 'Energy',
    310: 'Industrials',
    311: 'Technology' ,    
}



In [3]:
class MyCustomFactor(CustomFactor):
    def compute(self,today,assets,out,close,returns):
        out[:] = ((close[-21] - close[-252])/close[-252] -
                      (close[-1] - close[-21])/close[-21]) / np.nanstd(returns, axis=0)

In [ ]:
def make_pipeline():
    
    # Instantiate some values
    sector = Sector()
    universe = Q1500US()
    returns = Returns(window_length = 60)
    
    # Put your filters here
    adv = AverageDollarVolume(window_length=10)>5e8
    
    # Put your factors here
    price = USEquityPricing.close.latest
    sma = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=20,mask=universe)
    custom = MyCustomFactor(inputs=[USEquityPricing.close,returns],mask=universe,window_length=252)
    
    # Instanciate pipline
    pipe = Pipeline(screen = universe)
    
    # Add Factors and Filters to Pipeline
    pipe.add(USEquityPricing.close.latest,'price')
    pipe.add(adv,'adv')
    pipe.add(adv,'sma')
    pipe.add(sector,'sector')
    
    return pipe

In [ ]:
result = run_pipeline(make_pipeline(),'2015-01-01', '2016-01-01')

In [ ]:
my_factor = result['sma']
sectors = result['sector']

# Get prices and extend them to make sure we capture the whole period
prices = get_pricing(asset_list, start_date='2015-12-01', end_date='2016-02-01', fields='price')

In [ ]:
factor_data = al.utils.get_clean_factor_and_forward_returns(my_factor,
                                                                   prices,
                                                                   quantiles=5,
                                                                   groupby=sectors,
                                                                   groupby_labels=MORNINGSTAR_SECTOR_CODES,
                                                                   periods=(1,5,10,20,30))

In [ ]:
al.tears.create_full_tear_sheet(factor_data,by_group=True)